In [1]:
import os

In [7]:
%run RNN_Train_Horizon.ipynb

# Get Data

In [3]:
Stock_Code = 'SPY'

In [4]:
AnalysisPath =  f"{RNN_PATH}/{Stock_Code}/Analysis"


# Create directory if it doesn't exist
if not os.path.exists(AnalysisPath):

    os.makedirs(AnalysisPath)

print(f"Created an Analysis Folder for this! ")


Created an Analysis Folder for this! 


In [5]:
path = f'{SOURCE_PATH}/{Stock_Code}.csv'
df = pd.read_csv(path)[['Code','Date','Close']]
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)

print(f"There are {len(df)} days in total ")

There are 3992 days in total 


# Parameter List

## destination 

In [6]:
csv_path = f"{RNN_PATH}/{Stock_Code}/Analysis/Model_Result.csv"

In [7]:
if os.path.exists(csv_path):
    
    os.remove(csv_path)
    
    print(f"Deleted {csv_path}")

else:
    
    print(f"File {csv_path} does not exist")

File Result/RNN/SPY/Analysis/Model_Result.csv does not exist


## Target

In [8]:
Horizon = 10
H_Increm = 10

Max_H = 40

## Variable

In [9]:
# Length of the stock price data source
Df_Len = 800
Df_Len_M = len(df)
D_Increm = int((Df_Len_M - Df_Len) / 10)


# Proportion of training or validation dataset
Train_Fraction = 6
Max_T = 8
T_Increm = 1

# Number of layer of the neural network
Layer = 1
Max_L = 3
L_Increm = 1

# how many days for each prediction
Past_Len = Horizon
P_Gp = 10

# Function to save predictions result

In [10]:
def Save_Prediction(predict_result):
    
    predict_result = pd.DataFrame(predict_result)

    if os.path.exists(csv_path):
        
        new_data = pd.read_csv(csv_path) 
        predict_result = pd.concat([new_data , predict_result], ignore_index=True) 
    
    
    predict_result.to_csv(csv_path, index=False)


# Count total combination

In [11]:
import math

print ( "Total Combination = ",
    (
        math.ceil((Df_Len_M - Df_Len) / D_Increm) *
        (9 - Train_Fraction) *
        math.ceil((4 - Layer) / L_Increm) *
        math.ceil((30 - Horizon) / H_Increm) *
        (P_Gp + 1)
    )
)

Total Combination =  2178


# Main Loop

In [ ]:
Full_Result = []
id = 1


for d in range( int(len(df) * 3/4) , Df_Len,  - D_Increm):

    input_df = df[-d:].copy()

    scaler = MinMaxScaler((0,1))
    scaler.fit(input_df[['Close']])
    input_df['Scaled_Close'] = scaler.transform(input_df[['Close']])
    
    
    for t in range(Train_Fraction, Max_T + 1 , T_Increm):

        t = t/10

    
        for l in range(Layer, Max_L + 1,  L_Increm):

        
            for h in range(Horizon, Max_H,  H_Increm):

                max_p = int(len(input_df) / 4)

            
                for p in range(Past_Len, max_p, int((max_p - Past_Len)/P_Gp)  ):

                    intro_str = f"Data Len = {str(d).ljust(4)}, Train = {t}, Layer = {l}, Horizon = {str(h).ljust(2)}, Past_Length = {str(p).ljust(4)}"
                    
                    print(f"{(str(id) + ')').ljust(5)} Running Data with {intro_str} ")


    
                    if d > (p + h) * 2 :
                    
                        # Run RNN
                        mae, mae_model, model, data_df, indi_df = RNN(input_df, p , t , h, scaler)

                        
                        # Title and model name
                        RefName = f"RNN_{Stock_Code}_H{str(h).zfill(len(str(Max_H)))}_{id}"                        
                        title = f"{RefName}\n{intro_str}\nAggr mae = {round(mae,3)}\nIndi mae = {np.round(mae_model,3)}"    

                        
                        # plot the result
                        Plot_Result(data_df, title, Stock_Code, model = 'RNN' , name = f"Analysis/{RefName}" , IsSave = True, IsDisplay=False)


                        
                        # Save current result
                        Curr_Result = {
                            "id": id,
                            "Horizon": h,
                            "Data_Length":d,
                            "Layer": l,
                            "Train_Fraction": t,
                            "Seq_Len" : p,
                            "mae_aggr":mae,
                            "mae_indi":mae_model,
                            "RefName": RefName,    
                        }                        
    
                        Full_Result.append(Curr_Result)

                        
                        # Save the Intermediate Result
                        if id % 10 == 0:                             
                            
                            Save_Prediction(Full_Result)
                            
                            Full_Result = []

                            print("Intermediate Result Saved")
                    

                        
                        # Clear Memory
                        del model
                        tf.keras.backend.clear_session()
                        gc.collect()


                        
                        # Next Model
                        id += 1                    
                    
                    else:

                        print(f"Skipped for {intro_str}")
                        
                        continue
                    
                        
Save_Prediction(Full_Result)

print("End Of Loop")
